# DSNext concept

Extend DSN by having interplanetary replay station

In [45]:
import numpy as np
from scipy.integrate import ode


# propagator from poliastro
# https://poliastro.readthedocs.io/en/stable/api/safe/twobody/propagation.html
import poliastro
#from poliastro.twobody.propagation import cowell

In [46]:
# sun gravitational parameter
mu = 132712000000
au2km = 149597871

In [47]:
# assume circular orbits for earth and mars
# Earth ephemeris
def Earth(param):
    """Function returns ephemeris value prompted, as defined by GTOC4 problem statement
    Args:
        param (str): orbital parameter to be fetched
            a: semi-major axis [AU]
            e: eccentricity
            i: inclination [deg]
            LAN: longitude of ascendig node [deg]
            omega: argument of periapsis [deg]
            M0: mean anomaly [deg]
            t: Epoch at which these elements are defined [MJD]
    Returns:
        (float): value of prompted parameter
    """
    if param == 'a':
        retval = 1   #0.999988049532578
    elif param == 'e':
        retval = 0
    elif param == 'i':
        retval = 0
    elif param == 'LAN':
        retval = 0
    elif param == 'omega':
        retval = 0
    elif param == 'M0':
        retval = 0
    elif param == 't':
        retval = 54000
    else:
        print('function parameter is invalid')
        return None
    return retval

# Earth state-vector - FIXME!!
vp = np.sqrt(mu/(Earth('a')*au2km))
Earth_r0 = np.array([Earth('a')*au2km, 0, 0])
Earth_v0 = np.array([0, vp, 0])
print(f'Earth initial vector: e6*{Earth_r0/10**6} [km], \n                         {Earth_v0} [km/s]')

Earth initial vector: e6*[149.597871   0.         0.      ] [km], 
                         [ 0.         29.78464243  0.        ] [km/s]


In [48]:
# Mars ephemeris
def Mars(param):
    """Function returns ephemeris value prompted for Mars
    Args:
        param (str): orbital parameter to be fetched
            a: semi-major axis [AU]
            e: eccentricity
            i: inclination [deg]
            LAN: longitude of ascendig node [deg]
            omega: argument of periapsis [deg]
            M0: mean anomaly [deg]
            t: Epoch at which these elements are defined [MJD]
    Returns:
        (float): value of prompted parameter
    """
    if param == 'a':
        retval = 1.524
    elif param == 'e':
        retval = 0
    elif param == 'i':
        retval = 0
    elif param == 'LAN':
        retval = 0
    elif param == 'omega':
        retval = 0
    elif param == 'M0':
        retval = 0
    elif param == 't':
        retval = 54000
    else:
        print('function parameter is invalid')
        return None
    return retval

# Mars state-vector - FIXME!!
vp = np.sqrt(mu/(Mars('a')*au2km))
Mars_r0 = np.array([Mars('a')*au2km, 0, 0])
Mars_v0 = np.array([0, vp, 0])
print(f'Mars initial vector: e6*{Mars_r0/10**6} [km], \n                        {Mars_v0} [km/s]')

Mars initial vector: e6*[227.9871554   0.          0.       ] [km], 
                        [ 0.         24.12681017  0.        ] [km/s]


In [62]:
# propagate by Cowell's method
Mars_P = 2*np.pi*np.sqrt((Mars('a')*au2km)**3/mu) # seconds
print(f'Mars orbital period: {Mars_P/(60*60*24)} days')

# time step... eventually: [x*(etTwo-etOne)/step + etOne for x in range(step)] BUT MIGHT MESS WITH INTEGRATOR!!!
step = 100
tofs = np.array([x*(Mars_P*3)/step for x in range(step)])
dt = tofs[1] - tofs[0] # time-step
print(f'Time step size: {dt/(60*60*24)} days')
#Mars_rr, Mars_vv = cowell(mu, Mars_r0, Mars_v0, tofs)
#Mars_rr, Mars_vv = poliastro.twobody.propagation.cowell(mu, Mars_r0, Mars_v0, tofs)


Mars orbital period: 687.1896323987264 days
Time step size: 20.61568897196179 days


In [63]:
from poliastro.twobody.propagation import func_twobody
res = np.zeros((tofs.size, 6))
u0 = np.concatenate((Mars_r0, Mars_v0), axis=None)

res[0] = u0
ii = 1

accel = 1e-5

rr = ode(func_twobody).set_integrator('dop853')  # All parameters by default
rr.set_initial_value(u0, tofs[0])
rr.set_f_params(mu)#, constant_accel_factory(accel))

while rr.successful() and rr.t + dt < tofs[-1]:
    rr.integrate(rr.t + dt)
    res[ii] = rr.y
    ii += 1

res[:5]

error: failed in processing argument list for call-back fcn.